---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [50]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [51]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns=[]
    state=None

    #read file university_towns.txt line by line
    #each entry in the file either represents a state or a region
    #the entry of a state is followed by string [edit], use this info to parse states
    #the entries below a state entry, are the regions from that state
    
    with open('university_towns.txt', 'r') as txtfile:
        for line in txtfile:
            line = line.strip()
            line = line.strip('\n')
            if '[edit]' in line:
                line = line.split('[')[0]
                state = line
            else:
                if '(' in line:
                    region = line.split('(')[0].strip()
                else:
                    region = line
                
                university_towns.append([state,region])   
    
    return pd.DataFrame(university_towns, columns=["State", "RegionName"])

In [52]:
towns = get_list_of_university_towns()
print(towns.shape)
print()
print(towns.iloc[[1, -1]])

(517, 2)

       State RegionName
1    Alabama   Florence
516  Wyoming    Laramie


In [53]:
def get_GDP():
    GDP = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6], names=['quarter', 'gdp'])
    index_1999 = GDP[GDP['quarter']=='1999q4'].index[0]
    GDP = GDP.iloc[index_1999:]
    GDP = GDP.reset_index(drop=True)
    
    for i in range(len(GDP)):
        if(i==0):
            continue
        
        GDP.loc[i, 'growth'] = GDP.loc[i, 'gdp'] - GDP.loc[i-1, 'gdp']
        
    GDP.loc[0, 'growth']=100
    return GDP

In [54]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP = get_GDP()
    recession = []
    for i in range(1, len(GDP)-1):
        if GDP.loc[i-1, 'growth'] >= 0 and GDP.loc[i, 'growth'] < 0 and  GDP.loc[i+1, 'growth'] <0 :
            recession.append(GDP.iloc[i])
        
    return recession[0]['quarter']

In [55]:
get_recession_start()

'2008q3'

In [56]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    GDP = get_GDP()
    recession_start = get_recession_start()
    
    start_index = GDP[GDP['quarter']==recession_start].index[0]
    end_index = None
    
    for i in range(start_index+1, len(GDP)-1):
        if GDP.loc[i, 'growth'] >= 0 and GDP.loc[i+1, 'growth'] >= 0:
            end_index=i+1
            break
            
    
    return GDP.loc[end_index, 'quarter']

In [57]:
get_recession_end()

'2009q4'

In [58]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP = get_GDP()
    start_index = GDP[GDP['quarter'] == get_recession_start()].index[0]
    end_index = GDP[GDP['quarter'] == get_recession_end()].index[0]
    
    return GDP.iloc[start_index:end_index+1].sort_values(by=['gdp']).iloc[0, 0]

In [59]:
get_recession_bottom()

'2009q2'

In [60]:
def map_state_names(row):
    
    if row['State'] in states:
        state = row['State']
        row['State'] = states[state]
    return row   

In [61]:
def get_all_homes():
    
    columns = ['RegionName', 'State', '2000-01', '2000-02','2000-03','2000-04','2000-05',
               '2000-06','2000-07','2000-08','2000-09','2000-10','2000-11','2000-12',
               '2001-01','2001-02','2001-03','2001-04','2001-05','2001-06','2001-07',
               '2001-08','2001-09','2001-10','2001-11','2001-12','2002-01','2002-02',
               '2002-03','2002-04','2002-05','2002-06','2002-07','2002-08','2002-09',
               '2002-10','2002-11','2002-12','2003-01','2003-02','2003-03','2003-04',
               '2003-05','2003-06','2003-07','2003-08','2003-09','2003-10','2003-11',
               '2003-12','2004-01','2004-02','2004-03','2004-04','2004-05','2004-06',
               '2004-07','2004-08','2004-09','2004-10','2004-11','2004-12','2005-01',
               '2005-02','2005-03','2005-04','2005-05','2005-06','2005-07','2005-08',
               '2005-09','2005-10','2005-11','2005-12','2006-01','2006-02','2006-03',
               '2006-04','2006-05','2006-06','2006-07','2006-08','2006-09','2006-10',
               '2006-11','2006-12','2007-01','2007-02','2007-03','2007-04','2007-05',
               '2007-06','2007-07','2007-08','2007-09','2007-10','2007-11','2007-12',
               '2008-01','2008-02','2008-03','2008-04','2008-05','2008-06','2008-07',
               '2008-08','2008-09','2008-10','2008-11','2008-12','2009-01','2009-02',
               '2009-03','2009-04','2009-05','2009-06','2009-07','2009-08','2009-09',
               '2009-10','2009-11','2009-12','2010-01','2010-02','2010-03','2010-04',
               '2010-05','2010-06','2010-07','2010-08','2010-09','2010-10','2010-11',
               '2010-12','2011-01','2011-02','2011-03','2011-04','2011-05','2011-06',
               '2011-07','2011-08','2011-09','2011-10','2011-11','2011-12','2012-01',
               '2012-02','2012-03','2012-04','2012-05','2012-06','2012-07','2012-08',
               '2012-09','2012-10','2012-11','2012-12','2013-01','2013-02','2013-03',
               '2013-04','2013-05','2013-06','2013-07','2013-08','2013-09','2013-10',
               '2013-11','2013-12','2014-01','2014-02','2014-03','2014-04','2014-05',
               '2014-06','2014-07','2014-08','2014-09','2014-10','2014-11','2014-12',
               '2015-01','2015-02','2015-03','2015-04','2015-05','2015-06','2015-07',
               '2015-08','2015-09','2015-10','2015-11','2015-12','2016-01','2016-02',
               '2016-03','2016-04','2016-05','2016-06','2016-07','2016-08']
    
    homes = pd.read_csv('City_Zhvi_AllHomes.csv', usecols=columns)
    
    return homes           
               

In [62]:
def convert_to_quarters(row):
    quarters = ['2000q1', '2000q2', '2000q3', '2000q4',
                '2001q1', '2001q2', '2001q3', '2001q4',
                '2002q1', '2002q2', '2002q3', '2002q4',
                '2003q1', '2003q2', '2003q3', '2003q4',
                '2004q1', '2004q2', '2004q3', '2004q4',
                '2005q1', '2005q2', '2005q3', '2005q4',
                '2006q1', '2006q2', '2006q3', '2006q4',
                '2007q1', '2007q2', '2007q3', '2007q4',
                '2008q1', '2008q2', '2008q3', '2008q4',
                '2009q1', '2009q2', '2009q3', '2009q4',   
                '2010q1', '2010q2', '2010q3', '2010q4',
                '2011q1', '2011q2', '2011q3', '2011q4',
                '2012q1', '2012q2', '2012q3', '2012q4',
                '2013q1', '2013q2', '2013q3', '2013q4',
                '2014q1', '2014q2', '2014q3', '2014q4',
                '2015q1', '2015q2', '2015q3', '2015q4',
                '2016q1', '2016q2', '2016q3']
    result={}
    col = 0
    for i in range(len(quarters)):
        result[quarters[i]] = row.iloc[col:col+4].mean()
        col+=3
    
    return pd.Series(result)
        
    

In [63]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    homes = get_all_homes()
    homes = homes.apply(map_state_names, axis=1)
    homes.set_index(['State', 'RegionName'], inplace=True)
    homes.columns = pd.to_datetime(homes.columns)
    homes = homes.resample('Q', axis=1).mean()
    homes = homes.rename(columns=lambda x: str(x.to_period('Q')).lower())
    return homes

In [64]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    university_towns = get_list_of_university_towns()
    recession_start = get_recession_start()
    recession_bottom  = get_recession_bottom()
    housing = convert_housing_data_to_quarters()
    q_before_recession_start = housing.columns[housing.columns.get_loc(recession_start)-1]
    housing['priceratio'] = housing[q_before_recession_start].div(housing[recession_bottom])
    df = pd.merge(housing.reset_index(), university_towns, on=university_towns.columns.tolist(), how='outer', indicator='_flag')
    housing_university = df[df['_flag']=='both']
    housing_non_university = df[df['_flag']=='left_only']
    result = ttest_ind(housing_university['priceratio'], housing_non_university['priceratio'], nan_policy='omit')
    p = result.pvalue
    if p < 0.01:
        different=True
    else:
        different=False
    
    if housing_university['priceratio'].mean() < housing_non_university['priceratio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
        
    return (different, p, better)

In [37]:
run_ttest()

(True, 0.0030592742966576714, 'university town')

In [48]:
u_towns = get_list_of_university_towns()

In [49]:
u_towns[u_towns['State']=='Massachusetts']

,State,RegionName
175,Massachusetts,Boston
176,Massachusetts,Bridgewater
177,Massachusetts,Cambridge
178,Massachusetts,Chestnut Hill
179,Massachusetts,The Colleges of Worcester Consortium:
180,Massachusetts,Dudley
181,Massachusetts,North Grafton
182,Massachusetts,Paxton
183,Massachusetts,Worcester
184,Massachusetts,The Five College Region of Western Massachusetts:


In [65]:
run_ttest()

(True, 0.0027240637047531249, 'university town')